In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('/Users/tristannisbet/Documents/SM/survey_responses.csv')

In [3]:
df.head()

,Timestamp,What country are you from?,Age Range,Gender,1. Choose your top favorite 3-5 cities you have traveled to that are on this list. - Favorite City #1,2. Favorite city #2,3. Favorite city #3,4. Favorite city #4,5. Favorite city #5,"6. If there's a city you have been and loved that is not on this list, add it below.",...,"From the following types of tourist attractions, which ones are you likely to go do while traveling? [Aquarium]","From the following types of tourist attractions, which ones are you likely to go do while traveling? [Amusement Park ]","From the following types of tourist attractions, which ones are you likely to go do while traveling? [Zoo]","From the following types of tourist attractions, which ones are you likely to go do while traveling? [Movie Theater]","From the following types of tourist attractions, which ones are you likely to go do while traveling? [Mall / Souvenir shop ]","From the following types of tourist attractions, which ones are you likely to go do while traveling? [Park ]","From the following types of tourist attractions, which ones are you likely to go do while traveling? [Natural Feature / Beach]","From the following types of tourist attractions, which ones are you likely to go do while traveling? [Place of Worship (Church/Temple)]",ID,id
0,6/1/20 9:42,United States,26-40,Female,Krabi,Rome,Budapest,NaN,NaN,"Hakone, Japan",...,1 ( Would NOT go),1 ( Would NOT go),1 ( Would NOT go),2,4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),NaN,0.0
1,6/1/20 9:49,United States,15-25,Female,Cancun,Las Vegas,Los Angeles,New York City,London,NaN,...,2,3,3,3,3,3,3,1 ( Would NOT go),NaN,1.0
2,6/1/20 9:57,United States,15-25,Female,Sydney,London,Dublin,Prague,Rome,None,...,4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),3,NaN,2.0
3,6/1/20 10:16,United States,26-40,Female,Rome,London,Florence,Mexico City,Munich,Astoria,...,3,2,2,3,3,4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),NaN,NaN
4,6/1/20 10:17,United States,26-40,Female,Denpasar,Shanghai,Kyoto,New York City,Vancouver,NaN,...,1 ( Would NOT go),2,1 ( Would NOT go),3,2,4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),NaN,NaN


In [4]:
df.rename(columns = {'What country are you from? ': 'nationality', 'Age Range': 'age', 'Gender': 'gender',
                    '1. Choose your top favorite 3-5 cities you have traveled to that are on this list.  - Favorite City #1': 'favorite_city_one',
                    '2. Favorite city #2': 'favorite_city_two', '3. Favorite city #3': 'favorite_city_three',
                    '4. Favorite city #4': 'favorite_city_four', '5. Favorite city #5': 'favorite_city_five',
                    "6. If there's a city you have been and loved that is not on this list, add it below. ": 'extra_favorite',
                    "7. What cities on this list have you been to and not enjoyed?   - Least favorite city #1": 'least_favorite_one',
                    '8. Least favorite city #2': 'least_favorite_two', 
                    "9. If there's a city you have been to and haven't liked that is not on this list, add it below": 'extra_least_favorite',
                    "What price range of restaurant do you eat at when you travel? [Price level: 1 (Fast/Cheap Eats)]": 'food_one',
                    "What price range of restaurant do you eat at when you travel? [Price level: 2 (Casual Dining)]": 'food_two',
                    "What price range of restaurant do you eat at when you travel? [Price level: 3 (Upscale Dining)]": 'food_three',
                    "What price range of restaurant do you eat at when you travel? [Price level: 4 (Fine Dining/High End)]": 'food_four',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Art Gallery]": 'art_gallery',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Library]": 'library',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Museum ]": 'museum',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Aquarium]": 'aquarium',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Amusement Park ]": 'amusement_park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Zoo]": 'zoo',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Movie Theater]": 'movie_theater',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Mall / Souvenir shop ]": 'store',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Park ]": 'park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Natural Feature / Beach]": 'natural_feature',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Place of Worship (Church/Temple)]": 'place_of_worship'}, inplace=True )

In [5]:
df.columns

Index(['Timestamp', 'nationality', 'age', 'gender', 'favorite_city_one',
       'favorite_city_two', 'favorite_city_three', 'favorite_city_four',
       'favorite_city_five', 'extra_favorite', 'least_favorite_one',
       'least_favorite_two', 'extra_least_favorite', 'food_one', 'food_two',
       'food_three', 'food_four', 'art_gallery', 'library', 'museum',
       'aquarium', 'amusement_park', 'zoo', 'movie_theater', 'store', 'park',
       'natural_feature', 'place_of_worship', 'ID', 'id '],
      dtype='object')

In [6]:
df.fillna('None', inplace=True)

In [7]:
df.drop(columns=['Timestamp', 'extra_favorite', 'extra_least_favorite'], inplace=True)

In [8]:
df.columns

Index(['nationality', 'age', 'gender', 'favorite_city_one',
       'favorite_city_two', 'favorite_city_three', 'favorite_city_four',
       'favorite_city_five', 'least_favorite_one', 'least_favorite_two',
       'food_one', 'food_two', 'food_three', 'food_four', 'art_gallery',
       'library', 'museum', 'aquarium', 'amusement_park', 'zoo',
       'movie_theater', 'store', 'park', 'natural_feature', 'place_of_worship',
       'ID', 'id '],
      dtype='object')

In [9]:
df

,nationality,age,gender,favorite_city_one,favorite_city_two,favorite_city_three,favorite_city_four,favorite_city_five,least_favorite_one,least_favorite_two,...,aquarium,amusement_park,zoo,movie_theater,store,park,natural_feature,place_of_worship,ID,id
0,United States,26-40,Female,Krabi,Rome,Budapest,None,None,None,None,...,1 ( Would NOT go),1 ( Would NOT go),1 ( Would NOT go),2,4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),None,0
1,United States,15-25,Female,Cancun,Las Vegas,Los Angeles,New York City,London,None,None,...,2,3,3,3,3,3,3,1 ( Would NOT go),None,1
2,United States,15-25,Female,Sydney,London,Dublin,Prague,Rome,Milan,None,...,4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),3,None,2
3,United States,26-40,Female,Rome,London,Florence,Mexico City,Munich,Las Vegas,Dallas,...,3,2,2,3,3,4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),None,None
4,United States,26-40,Female,Denpasar,Shanghai,Kyoto,New York City,Vancouver,Manila,None,...,1 ( Would NOT go),2,1 ( Would NOT go),3,2,4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Canada,15-25,Male,Melbourne,Vancouver,Sydney,None,None,Tokyo,None,...,3,3,2,1 ( Would NOT go),2,3,3,4 (Definitely would go),None,None
149,United States,26-40,Male,Atlanta,New York City,Cairo,Honolulu,San Francisco,None,None,...,4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),2,2,1 ( Would NOT go),None,None
150,United States,26-40,Male,Berlin,Los Angeles,Istanbul,None,None,New York City,None,...,2,1 ( Would NOT go),2,1 ( Would NOT go),1 ( Would NOT go),3,4 (Definitely would go),2,None,None
151,United States,26-40,Male,Florence,Miami,Las Vegas,None,None,Vancouver,None,...,2,2,1 ( Would NOT go),1 ( Would NOT go),3,3,4 (Definitely would go),3,None,None


In [10]:
city_df = df.copy()

In [11]:
city_df.drop(columns=['nationality', 'age', 'gender'], inplace=True)
city_df

,favorite_city_one,favorite_city_two,favorite_city_three,favorite_city_four,favorite_city_five,least_favorite_one,least_favorite_two,food_one,food_two,food_three,...,aquarium,amusement_park,zoo,movie_theater,store,park,natural_feature,place_of_worship,ID,id
0,Krabi,Rome,Budapest,None,None,None,None,Always,Always,Rarely,...,1 ( Would NOT go),1 ( Would NOT go),1 ( Would NOT go),2,4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),None,0
1,Cancun,Las Vegas,Los Angeles,New York City,London,None,None,Sometimes,Often,Sometimes,...,2,3,3,3,3,3,3,1 ( Would NOT go),None,1
2,Sydney,London,Dublin,Prague,Rome,Milan,None,Sometimes,Often,Sometimes,...,4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),3,None,2
3,Rome,London,Florence,Mexico City,Munich,Las Vegas,Dallas,Sometimes,Often,Sometimes,...,3,2,2,3,3,4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),None,None
4,Denpasar,Shanghai,Kyoto,New York City,Vancouver,Manila,None,Often,Often,Rarely,...,1 ( Would NOT go),2,1 ( Would NOT go),3,2,4 (Definitely would go),4 (Definitely would go),1 ( Would NOT go),None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Melbourne,Vancouver,Sydney,None,None,Tokyo,None,Sometimes,Often,Rarely,...,3,3,2,1 ( Would NOT go),2,3,3,4 (Definitely would go),None,None
149,Atlanta,New York City,Cairo,Honolulu,San Francisco,None,None,Rarely,Often,Sometimes,...,4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),4 (Definitely would go),2,2,1 ( Would NOT go),None,None
150,Berlin,Los Angeles,Istanbul,None,None,New York City,None,Sometimes,Often,Rarely,...,2,1 ( Would NOT go),2,1 ( Would NOT go),1 ( Would NOT go),3,4 (Definitely would go),2,None,None
151,Florence,Miami,Las Vegas,None,None,Vancouver,None,Often,Always,Sometimes,...,2,2,1 ( Would NOT go),1 ( Would NOT go),3,3,4 (Definitely would go),3,None,None


In [12]:
city_df.reset_index(drop=False, inplace=True)
city_df.rename(columns={'index': 'user_id'}, inplace=True)

In [13]:
dummy = pd.get_dummies(df)
dummy_city = pd.get_dummies(city_df)

In [14]:
dummy

,nationality_Australia,nationality_Canada,nationality_China,nationality_Finland,nationality_Honduras,nationality_India,nationality_Israel,nationality_Japan,nationality_Mexico,nationality_Pakistan,...,natural_feature_4 (Definitely would go),place_of_worship_1 ( Would NOT go),place_of_worship_2,place_of_worship_3,place_of_worship_4 (Definitely would go),ID_None,id _0.0,id _1.0,id _2.0,id _None
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
149,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
150,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
151,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,1


In [15]:
dummy_city

,user_id,favorite_city_one_Amsterdam,favorite_city_one_Atlanta,favorite_city_one_Barcelona,favorite_city_one_Beijing,favorite_city_one_Berlin,favorite_city_one_Boston,favorite_city_one_Budapest,favorite_city_one_Cancun,favorite_city_one_Chicago,...,natural_feature_4 (Definitely would go),place_of_worship_1 ( Would NOT go),place_of_worship_2,place_of_worship_3,place_of_worship_4 (Definitely would go),ID_None,id _0.0,id _1.0,id _2.0,id _None
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0
2,2,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1
4,4,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,148,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
149,149,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
150,150,0,0,0,0,1,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
151,151,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,1


In [ ]:
# Not using one hot encoder..
#enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
#X_enc = enc.fit_transform(df)
#X_enc

In [16]:
cosine = cosine_similarity(dummy)
cosine_city = cosine_similarity(dummy_city)

In [17]:
# city does not have user demographics
cosine_df = pd.DataFrame(cosine)
cosine_df_city = pd.DataFrame(cosine_city)

In [18]:
cosine_df_city

,0,1,2,3,4,5,6,7,8,9,...,143,144,145,146,147,148,149,150,151,152
0,1.000000,0.163299,0.231455,0.106600,0.258199,0.204124,0.026352,0.167236,0.087039,0.159364,...,0.014266,0.008500,0.009849,0.012576,0.009715,0.009649,0.005477,0.010881,0.010809,0.008053
1,0.163299,1.000000,0.264575,0.382971,0.316228,0.457143,0.438938,0.327715,0.298481,0.273252,...,0.209667,0.206825,0.206779,0.205364,0.212127,0.209345,0.207941,0.207889,0.207838,0.203842
2,0.231455,0.264575,1.000000,0.427669,0.448211,0.404962,0.414758,0.464493,0.483494,0.534841,...,0.385668,0.388239,0.384262,0.382927,0.385464,0.386691,0.389169,0.390355,0.390275,0.385224
3,0.106600,0.382971,0.427669,1.000000,0.578006,0.571969,0.606780,0.611227,0.612372,0.560612,...,0.531660,0.534013,0.530334,0.526706,0.527861,0.529000,0.531292,0.528914,0.528872,0.531119
4,0.258199,0.316228,0.448211,0.578006,1.000000,0.609868,0.612372,0.647704,0.623635,0.725226,...,0.639820,0.639771,0.640813,0.639676,0.638555,0.641719,0.639538,0.640547,0.641542,0.641484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,0.009649,0.209345,0.386691,0.529000,0.641719,0.727401,0.781147,0.825163,0.855216,0.884420,...,0.999244,0.999250,0.999255,0.999122,0.999311,1.000000,0.999139,0.999505,0.999329,0.999245
149,0.005477,0.207941,0.389169,0.531292,0.639538,0.719649,0.779374,0.822771,0.859493,0.881763,...,0.999061,0.999161,0.999029,0.999081,0.999179,0.999139,1.000000,0.999151,0.999156,0.999206
150,0.010881,0.207889,0.390355,0.528914,0.640547,0.722472,0.780205,0.827430,0.855190,0.886946,...,0.999300,0.999259,0.999265,0.999179,0.999366,0.999505,0.999151,1.000000,0.999515,0.999343
151,0.010809,0.207838,0.390275,0.528872,0.641542,0.721475,0.779316,0.826604,0.855882,0.888829,...,0.999166,0.999310,0.999224,0.999184,0.999415,0.999329,0.999156,0.999515,1.000000,0.999217


In [19]:
def find_similar_n(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [20]:
top_10 = find_similar_n(cosine_df,10)
top_10_c= find_similar_n(cosine_df_city,10)

In [21]:
top_10_c

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
0,0,4,2,5,7,1,9,10,13,12
1,1,5,6,3,11,12,7,4,8,15
2,2,9,18,8,17,7,19,22,4,29
3,3,8,7,11,6,16,20,19,21,4
4,4,9,10,17,19,20,13,23,12,14
...,...,...,...,...,...,...,...,...,...,...
148,148,150,151,138,147,142,130,145,144,152
149,149,131,137,139,152,120,140,147,138,144
150,150,138,151,148,134,139,135,133,142,147
151,151,150,138,147,136,142,134,133,139,148


In [22]:
top_10

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
0,0,47,38,22,101,88,143,86,79,18
1,1,53,41,87,92,5,11,61,94,97
2,2,64,42,30,9,105,120,46,72,17
3,3,99,85,66,67,125,126,43,32,53
4,4,20,56,53,101,23,115,126,17,125
...,...,...,...,...,...,...,...,...,...,...
148,148,150,5,94,71,34,42,73,26,43
149,149,131,64,75,22,35,53,58,46,8
150,150,138,151,34,73,9,128,17,133,101
151,151,9,17,34,150,138,73,79,62,42


In [23]:
df.iloc[[149, 130, 110, 105, 0, 54], 3:9]

,favorite_city_one,favorite_city_two,favorite_city_three,favorite_city_four,favorite_city_five,least_favorite_one
149,Atlanta,New York City,Cairo,Honolulu,San Francisco,None
130,Frankfurt,Honolulu,Boston,Vancouver,Chicago,Atlanta
110,London,Edinburgh,Rome,Melbourne,Munich,Atlanta
105,Dammam,London,Vienna,Amsterdam,Athens,Athens
0,Krabi,Rome,Budapest,None,None,None
54,Seoul,Osaka,Kyoto,Quito,Jeju,New York City


In [24]:
df = city_df[['user_id', 'favorite_city_one', 'favorite_city_two', 'favorite_city_three', 'favorite_city_four', 'favorite_city_five']].copy()

In [25]:
df

,user_id,favorite_city_one,favorite_city_two,favorite_city_three,favorite_city_four,favorite_city_five
0,0,Krabi,Rome,Budapest,None,None
1,1,Cancun,Las Vegas,Los Angeles,New York City,London
2,2,Sydney,London,Dublin,Prague,Rome
3,3,Rome,London,Florence,Mexico City,Munich
4,4,Denpasar,Shanghai,Kyoto,New York City,Vancouver
...,...,...,...,...,...,...
148,148,Melbourne,Vancouver,Sydney,None,None
149,149,Atlanta,New York City,Cairo,Honolulu,San Francisco
150,150,Berlin,Los Angeles,Istanbul,None,None
151,151,Florence,Miami,Las Vegas,None,None


In [27]:
df_backwards = df.rename(columns={'favorite_city_one': 5, 'favorite_city_two': 4, 'favorite_city_three': 3,
                  'favorite_city_four': 2, 'favorite_city_five': 1})
df_backwards

,user_id,5,4,3,2,1
0,0,Krabi,Rome,Budapest,None,None
1,1,Cancun,Las Vegas,Los Angeles,New York City,London
2,2,Sydney,London,Dublin,Prague,Rome
3,3,Rome,London,Florence,Mexico City,Munich
4,4,Denpasar,Shanghai,Kyoto,New York City,Vancouver
...,...,...,...,...,...,...
148,148,Melbourne,Vancouver,Sydney,None,None
149,149,Atlanta,New York City,Cairo,Honolulu,San Francisco
150,150,Berlin,Los Angeles,Istanbul,None,None
151,151,Florence,Miami,Las Vegas,None,None


In [28]:
df.rename(columns={'favorite_city_one': 1, 'favorite_city_two': 2, 'favorite_city_three': 3,
                  'favorite_city_four': 4, 'favorite_city_five': 5}, inplace=True)

In [29]:
melted = pd.melt(df, id_vars = ['user_id'], value_vars = [1,2,3,4,5])
melted_b = pd.melt(df, id_vars = ['user_id'], value_vars = [5,4,3,2,1])
#price_level = city_df.pivot_table(index=['country', 'city', 'id'], columns='price_level', values='name', aggfunc='first')
#pd.melt(df, id_vars =['Name'], value_vars =['Course']) 


In [31]:
melted

,user_id,variable,value
0,0,1,Krabi
1,1,1,Cancun
2,2,1,Sydney
3,3,1,Rome
4,4,1,Denpasar
...,...,...,...
760,148,5,None
761,149,5,San Francisco
762,150,5,None
763,151,5,None


In [32]:
melted.rename(columns={'value': 'city', 'variable':'rank'}, inplace=True)
melted = melted[['user_id', 'city', 'rank']]

melted_b.rename(columns={'value': 'city', 'variable':'rank'}, inplace=True)
melted_b = melted[['user_id', 'city', 'rank']]


In [33]:
melted.sort_values('user_id', inplace=True)
melted.reset_index(drop=True, inplace=True)

melted_b.sort_values('user_id', inplace=True)
melted_b.reset_index(drop=True, inplace=True)

In [34]:

melted['rank'] = pd.to_numeric(melted['rank'])
melted_b['rank'] = pd.to_numeric(melted['rank'])


In [50]:
melted_b.drop(melted_b[melted_b.city == 'None'].index, inplace=True)
melted.drop(melted[melted.city == 'None'].index, inplace=True)


In [51]:
melted

,user_id,city,rank
0,0,Krabi,1
1,0,Budapest,3
2,0,Rome,2
5,1,Las Vegas,2
6,1,New York City,4
...,...,...,...
760,152,Paris,3
761,152,Istanbul,2
762,152,Amsterdam,1
763,152,Barcelona,4


In [52]:
#suprise_5 = melted.to_csv(r"/Users/tristannisbet/Documents/SM/Dataframe/suprise_5.csv", index=False)
#suprise_b = melted_b.to_csv(r"/Users/tristannisbet/Documents/SM/Dataframe/suprise_b.csv", index=False)


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import KFold
from surprise import accuracy


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
melted.iloc[:, 1] = labelencoder.fit_transform(melted.iloc[:, 1])

In [ ]:
# Change it back to countries
melted.iloc[:, 1] = labelencoder.inverse_transform(melted.iloc[:, 1])

In [ ]:
reader= Reader(rating_scale=(1.0, 3.0))
data= Dataset.load_from_df(melted[['user_id', 'city', 'rank']], reader)


In [ ]:
melted

In [ ]:
kf = KFold(n_splits=3)
algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
mix = melted[['user_id', 'rank', 'city']].copy()

In [ ]:
mix.sort_values('city')

In [ ]:
reader2 = Reader(rating_scale=(1.0, 80.0))
data2 = Dataset.load_from_df(melted[['user_id', 'rank', 'city']], reader)

In [ ]:
kf = KFold(n_splits=3)
algo = SVD()

for trainset, testset in kf.split(data2):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

# Use movielens-100K
#data = Dataset.load_builtin('ml-100k')

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])